In [2]:
import os
from django.conf import settings
# settings.configure({
#     'DEBUG': 'True',
#     'AUTH_USER_MODEL': 'accounts.MyUser',
#     'AUTHENTICATION_BACKENDS': [
#         'accounts.backends.EmailAuthenticationBackend'
#     ]
# })
from feed import models as feed_models
from accounts import models
from dms import models as dms_models
from django.db.models import BooleanField
from django.db.models.expressions import Q, F, Case, When

In [3]:
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

In [4]:
users = models.MyUser.objects.all()
user1 = users.first()
user2 = users.get(username='kylie')

In [5]:
profile1 = user1.myuserprofile
profile2 = user2.myuserprofile
profile2

<MyUserProfile: kylie.jenner@gmail.com>

In [6]:
followers = profile1.followed_by.all()
followers

<QuerySet [<MyUserProfile: kylie.jenner@gmail.com>, <MyUserProfile: constance.jablonski@gmail.com>]>

In [7]:
profile2 in followers

True

In [8]:
constance = users.get(username='constance')
# constance.myuserprofile.follows.add(profile1)
constance.myuserprofile.follows.all()

<QuerySet [<MyUserProfile: pendenquejohn@gmail.com>, <MyUserProfile: kylie.jenner@gmail.com>]>

In [9]:
# constance.myuserprofile.follows.remove(profile1)
constance.myuserprofile.follows.all()

<QuerySet [<MyUserProfile: pendenquejohn@gmail.com>, <MyUserProfile: kylie.jenner@gmail.com>]>

In [10]:
comments = feed_models.Conversation.objects.all()

In [11]:
comment = comments.first()
comment

<Conversation: constance>

In [12]:
likes = comment.like_set.all()
print(likes, likes.count())

<QuerySet []> 0


In [13]:
user_in_likes = When(like__user__username='zadigo', then=True)
cases = Case(user_in_likes, default=False, output_field=BooleanField())
annotated_comments = comments.annotate(has_liked=cases)
annotated_comments.values_list('has_liked')

<QuerySet [(False,), (False,), (True,), (False,), (False,), (True,), (True,), (False,), (False,)]>

In [14]:
zadigo = users.get(username='zadigo')

In [26]:
messages = dms_models.DirectMessage.objects.filter(Q(message_sender=zadigo.id) | Q(message_receiver=zadigo.id))
messages

<QuerySet [<DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: zadigo>, <DirectMessage: constance>]>

In [27]:
messages.filter(Q(message_receiver__username=constance.username) | Q(message_sender__username=constance.username))

<QuerySet [<DirectMessage: zadigo>, <DirectMessage: constance>]>

In [38]:
list_of_users = messages.values_list('message_receiver__username', flat=True)
set(user for user in list(list_of_users) if user != "zadigo")

{'constance', 'kylie'}